In [1]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
import csv
from lib import *
%matplotlib inline

In [2]:
import os
os.environ['THEANO_FLAGS']='mode=FAST_RUN,device=gpu,lib.cnmem=0.7,floatX=float32'

## Bundle model begins

In [3]:
folder = '../data/new_data'
user_item_map=pickle.load(open(folder + '/user_item_map.pkl','rb'))
sgd_tags = pickle.load(open(folder + '/sgd_tags.pkl','rb'))
user_bundle_map=pickle.load(open(folder +'/user_bundle_map.pkl','rb'))
bundle_discount_map = pickle.load(open(folder +'/bundle_discount_map.pkl','rb'))
bundle_item_map=pickle.load(open(folder + '/bundle_item_map.pkl','rb'))
all_data=[]

for user,bundles in user_bundle_map.items():
    for bundle in bundles:
        all_data.append((user,bundle))

In [ ]:
item_data_map=dict() #appid, item
tags_set=set()
for item in item_data:
    item_data_map[int(item['item_id'])]=item
    for tag in item['genre']:
        tags_set.add(tag)
tags_map=dict()
for i,tag in enumerate(tags_set):
    tags_map[tag]=i 
    
def get_feat(tags):
    feat=np.zeros(len(tags_map))
    for tag in tags:
        feat[tags_map[tag]]=1
    return feat
    
def get_item_data_map_id(index):
    id = item_id_lookup[index]
    for i in range(0,len(item_data)-1):
        if item_data[i]['item_id'] == id:
            return i
            
sgd_tags = np.array([get_feat(item_data[get_item_data_map_id(i)]['genre']) for i in user_item_map])
print(sgd_tags.shape)

In [4]:
num_items = max([ max(user_item_map[user]) for user in user_item_map]) + 1
print num_items

2515


In [5]:
import random
random.shuffle(all_data)
training_data=all_data[:int(0.9*len(all_data))]
test_data=all_data[int(0.9*len(all_data)):]

In [6]:
# Generting training data for bundles through graph sampling.
sgd_users, sgd_pos_bundles, sgd_neg_bundles = graph_sampling(len(training_data)*30, training_data, user_bundle_map)

0


In [7]:
max_bundle_size = np.max([len(bundle_item_map[key]) for key in bundle_item_map])

In [8]:
def get_items(bundle_id, max_bundle_size, index):
    item=list(bundle_item_map[bundle_id]);
    for i in range(len(item),max_bundle_size):
        item.append(index)
    return item

sgd_pos_items=[get_items(b_id, max_bundle_size, num_items) for b_id in sgd_pos_bundles]
sgd_neg_items=[get_items(b_id, max_bundle_size, num_items) for b_id in sgd_neg_bundles]

In [9]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics.pairwise import cosine_similarity
def compute_diversity_tags(app_data):
    l=len(app_data)
    app_data=[item_id_lookup[d] for d in app_data]
    count=0.0
    similarity=0.0
    for i in range(l):
        if app_data[i] in item_data_map:
            for j in range(i+1,l):
                if app_data[j] in item_data_map:
                    count+=1
                    similarity+=jaccard_similarity_score(get_feat(item_data_map[app_data[i]]['tags']),
                                                         get_feat(item_data_map[app_data[j]]['tags']))
    if count>0:
        return 1-(similarity/count)
    else:
        return 0.0

def compute_diversity_latent(items, H):
    count=0.0
    similarity=0.0
    for i in range(len(items)):
        for j in range(i + 1, len(items)):
            itemID1, itemID2 = items[i], items[j]
            count+=1
            similarity+=cosine_similarity(H[itemID1].reshape(1,-1),H[itemID2].reshape(1,-1))[0,0]
    if count>0:
        return 1-(similarity/count)
    else:
        return 0.0
    
def compute_diversity(app_data, H):
    if H is not None:
        return compute_diversity_latent(app_data, H)
    else :
        return compute_diversity_tags(app_data)

In [10]:
bpr_item = pickle.load(open(folder +'/bpr_item.pkl','rb'))

In [11]:
H_item=bpr_item.H.eval()
B_item=bpr_item.B.eval()
W_item=bpr_item.W.eval()
A_item=bpr_item.A.eval()

In [12]:
bundle_diversity_map=dict()
for bundle,items in bundle_item_map.items():
    bundle_diversity_map[bundle]=compute_diversity_latent(list(items), H_item)

In [13]:
sgd_pos_len=[len(bundle_item_map[b_id]) for b_id in sgd_pos_bundles]
sgd_neg_len=[len(bundle_item_map[b_id]) for b_id in sgd_neg_bundles]
sgd_pos_diversity=[bundle_diversity_map[b_id] for b_id in sgd_pos_bundles]
sgd_neg_diversity=[bundle_diversity_map[b_id] for b_id in sgd_neg_bundles]
sgd_pos_dicount=[bundle_discount_map[b_id] for b_id in sgd_pos_bundles]
sgd_neg_dicount=[bundle_discount_map[b_id] for b_id in sgd_neg_bundles]

In [14]:
print len(sgd_neg_dicount), len(sgd_pos_diversity), len(sgd_pos_dicount)

2363790 2363790 2363790


In [15]:
def get_test_data_bundles1(test_data, train_data, n_items):
    users=[]
    pos_items=[]
    neg_items=[]
    n1=[]
    n2=[]
    pos_diversity, neg_diversity=[], []
    test_bd_pos_discount, test_bd_neg_discount = [], []
    train_dict, train_users, train_items  = data_to_dict(train_data)
    test_dict, test_users, test_items = data_to_dict(test_data)
    auc_values = []
    z = 0
    for i,user in enumerate(test_dict.keys()):
        if(i%100000==0):
            print i

        if user in train_users: 
            for pos_item in test_dict[user]:
                if pos_item in train_items:
                    for neg_item in train_items:
                        if neg_item not in test_dict[user] and neg_item not in train_dict[user]:
                            pos_diversity.append(bundle_diversity_map[pos_item])
                            neg_diversity.append(bundle_diversity_map[neg_item])
                            users.append(user)
                            pos_items.append(pos_item)
                            neg_items.append(neg_item)
                            n1.append(len(bundle_item_map[pos_item]))
                            n2.append(len(bundle_item_map[neg_item]))
                            test_bd_pos_discount.append(bundle_discount_map[pos_item])
                            test_bd_neg_discount.append(bundle_discount_map[neg_item])
                            

    pos_bunldes = pos_items
    neg_bundles = neg_items
    pos_items=[get_items(b_id, max_bundle_size, n_items) for b_id in pos_bunldes]
    neg_items=[get_items(b_id, max_bundle_size, n_items) for b_id in neg_bundles]
    return users, pos_items, neg_items, n1, n2, pos_diversity, neg_diversity, \
            test_bd_pos_discount, test_bd_neg_discount


In [16]:
test_users, test_pos_items, test_neg_items, test_n1, test_n2, test_pos_diversity, test_neg_diversity,\
test_bd_pos_discount, test_bd_neg_discount=\
                    get_test_data_bundles1(test_data, training_data, num_items)

0


In [17]:
print test_n1[0]
print type(H_item)

3
<type 'numpy.ndarray'>


In [18]:
H_item = np.concatenate((H_item,np.zeros((1,np.shape(H_item)[1]))),axis=0)
H_item=np.array(H_item).astype('float32')
print np.shape(H_item)


B_item = np.append(B_item,0)
B_item=np.array(B_item).astype('float32')
print np.shape(B_item)

A_item = np.concatenate((A_item,np.zeros((1,np.shape(A_item)[1]))),axis=0)
A_item=np.array(A_item).astype('float32')
print np.shape(A_item)

(2516, 10)
(2516,)
(2516, 327)


In [ ]:
class BPR_Cold(object):

    def __init__(self, rank, bundle_size, n_users, n_items, W_item, H_item, B_item, lambda_u = 0.0025, lambda_i = 0.0025, lambda_j = 0.00025, lambda_d = 0.0025, lambda_p = 0.00025, lambda_bias = 0.0, learning_rate = 0.05):
        
        self._rank = rank
        self._bundle_rank = bundle_size + 1
        self._n_users = n_users
        self._n_items = n_items
        self._lambda_u = lambda_u
        self._lambda_i = lambda_i
        self._lambda_j = lambda_j
        self._lambda_d = lambda_d
        self._lambda_p = lambda_p
        self.W1 = W_item
        self.H1 = theano.shared(H_item.astype('float32'), name='H')
        self.B1 = theano.shared(B_item.astype('float32'), name='B')
        self._lambda_bias = lambda_bias
        self._learning_rate = learning_rate
        self._configure_theano()
        self._generate_train_model_item_function()
        self._generate_test_model_function()

    def _configure_theano(self):
        theano.config.mode = 'FAST_RUN'
        theano.config.floatX = 'float32'
    
    def _generate_train_model_item_function(self):
        u = T.lvector('u')
        i = T.lmatrix('i')
        j = T.lmatrix('j')
        n1 = T.lvector('n1')
        n2 = T.lvector('n2')
        di = T.dvector('di')
        dj = T.dvector('dj')
        
        
        self.M1 = theano.shared(numpy.random.random((self._rank, self._rank)).astype('float64'), name='M1')
        self.M2 = theano.shared(numpy.random.random((self._rank, self._rank)).astype('float64'), name='M2')
        self.K = theano.shared(numpy.random.rand(), name='K')
        self.D = theano.shared(numpy.random.rand(), name='D')
        self.N = theano.shared(numpy.random.random(self._bundle_rank).astype('float32'), name='N')
        
        x_ui = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[i].sum(axis=1).T/n1)).diagonal() + self.K*(self.B1[i].T/n1).T.sum(axis=1) + self.N[n1] + self.D*di
        x_uj = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[j].sum(axis=1).T/n2)).diagonal() + self.K*(self.B1[j].T/n2).T.sum(axis=1) + self.N[n2] + self.D*dj
        
        x_uij = T.nnet.sigmoid(x_ui-x_uj)
        obj = T.sum(T.log(x_uij) - self._lambda_u * (self.M1 ** 2).sum() - \
                    self._lambda_u * (self.M2 ** 2).sum()  - self._lambda_d * (self.K**2) - self._lambda_d * (self.D**2)\
                    -self._lambda_p * (self.N[n2]**2) - self._lambda_p * (self.N[n1]**2))
        cost = - obj

        g_cost_M1 = T.grad(cost=cost, wrt=self.M1)
        g_cost_M2 = T.grad(cost=cost, wrt=self.M2)
        g_cost_K = T.grad(cost=cost, wrt=self.K)
        g_cost_N = T.grad(cost=cost, wrt=self.N)
        g_cost_D = T.grad(cost=cost, wrt=self.D)
        
        updates = [(self.M1, self.M1 - self._learning_rate * .001* g_cost_M1), (self.M2, self.M2 - self._learning_rate *.001* g_cost_M2), 
                   (self.K, self.K - self._learning_rate * .001*g_cost_K), (self.N, self.N - self._learning_rate *g_cost_N),
                  (self.D, self.D - self._learning_rate * g_cost_D)]

        self.train_model_item = theano.function(inputs=[u, i, j, n1, n2, di, dj], outputs=cost, updates=updates)

    
    def train(self, s_users=None, s_pos_items=None, s_neg_items=None, s_pos_len=None, s_neg_len=None,
             s_pos_diversity=None, s_neg_diversity=None,batch_size=1000):
        
        if len(s_users) < batch_size:
            sys.stderr.write("WARNING: Batch size is greater than number of training samples, switching to a batch size of %s\n" % str(len(train_data)))
            batch_size = len(s_users)
        
        sgd_users, sgd_pos_items, sgd_neg_items = s_users, s_pos_items, s_neg_items
        n_sgd_samples = len(s_users)

        z = 0
        t2 = t1 = t0 = time.time()
        while (z+1)*batch_size < n_sgd_samples:
            
            self.train_model_item(
                sgd_users[z*batch_size: (z+1)*batch_size],
                sgd_pos_items[z*batch_size: (z+1)*batch_size],
                sgd_neg_items[z*batch_size: (z+1)*batch_size],
                s_pos_len[z*batch_size: (z+1)*batch_size],
                s_neg_len[z*batch_size: (z+1)*batch_size],
                s_pos_diversity[z*batch_size: (z+1)*batch_size],
                s_neg_diversity[z*batch_size: (z+1)*batch_size],
            )
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            sys.stderr.flush()
            t1 = t2
        if n_sgd_samples > 0:
            sys.stderr.write("\nTotal training time %.2f seconds; %e per sample\n" % (t2 - t0, (t2 - t0)/n_sgd_samples))
            sys.stderr.flush()
    
    def _generate_test_model_function(self):
        u = T.lvector('u')
        i = T.lmatrix('i')
        j = T.lmatrix('j')
        n1 = T.lvector('n1')
        n2 = T.lvector('n2')
        di = T.dvector('di')
        dj = T.dvector('dj')
        
        x_ui = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[i].sum(axis=1).T/n1)).diagonal() + self.K*(self.B1[i].T/n1).T.sum(axis=1) + self.N[n1] + self.D*di
        x_uj = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[j].sum(axis=1).T/n2)).diagonal() + self.K*(self.B1[j].T/n2).T.sum(axis=1) + self.N[n2] + self.D*dj 
        
        x_uij = x_ui-x_uj
        self.test_model = theano.function(inputs=[u, i, j, n1, n2, di, dj], outputs=x_uij)
        
    def test_bundle(self, sgd_users, sgd_pos_items, sgd_neg_items, s_pos_len, s_neg_len, s_pos_diversity, s_neg_diversity, batch_size=1000):
        
        auc_values = []
        z = 0
        t2 = t1 = t0 = time.time()
        n_sgd_samples = len(sgd_users)
        while (z+1)*batch_size < n_sgd_samples:
            pref_list=self.test_model(
                sgd_users[z*batch_size: (z+1)*batch_size],
                sgd_pos_items[z*batch_size: (z+1)*batch_size],
                sgd_neg_items[z*batch_size: (z+1)*batch_size],
                s_pos_len[z*batch_size: (z+1)*batch_size],
                s_neg_len[z*batch_size: (z+1)*batch_size],
                s_pos_diversity[z*batch_size: (z+1)*batch_size],
                s_neg_diversity[z*batch_size: (z+1)*batch_size]
            )
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            t1 = t2
            
            auc = np.sum([1.0 if a>0.0 else 0.0 for a in pref_list])
            auc /= batch_size
            
            auc_values.append(auc)
            sys.stderr.write("\rCurrent AUC mean (%s samples): %0.5f" % (str(z*batch_size), numpy.mean(auc_values)))
            sys.stderr.flush()
        
        sys.stderr.write("\n")
        sys.stderr.flush()
        return numpy.mean(auc_values)

In [ ]:
bpr_cold = BPR_Cold(10, max_bundle_size, len(user_bundle_map.keys()), num_items, W_item, H_item, B_item)

In [ ]:
bpr_cold.train(s_users=sgd_users, s_pos_items=sgd_pos_items, s_neg_items=sgd_neg_items, 
         s_pos_len=sgd_pos_len, s_neg_len=sgd_neg_len, s_pos_diversity=sgd_pos_diversity,
               s_neg_diversity=sgd_neg_diversity)

In [ ]:
bpr_cold.test_bundle(test_users, test_pos_items, test_neg_items, test_n1, test_n2, test_pos_diversity, test_neg_diversity)

In [ ]:
print '0.68585732689210954'
bundle_discount_array = [bundle_discount_map[i] for i in range(len(bundle_discount_map)) if i ]

In [ ]:
print 0.73830120101137786 + 'discount devrivate'
print 0.72651723027375203 + 'no discount derivaite'

In [19]:
bunlde_max_id = max([x for x in sgd_pos_bundles])

In [24]:
class BPR_Buncle1(object):

    def __init__(self, rank, bundle_size, n_users, n_items, n_tags, bunlde_max_id, lambda_u = 0.0025, lambda_i = 0.0025, lambda_j = 0.00025, lambda_d = 0.0025, lambda_p = 0.00025, lambda_A = 0.01, lambda_bias = 0.0, learning_rate = 0.05):
        
        self._rank = rank
        self._bundle_rank = bundle_size + 1
        self._n_users = n_users
        self._n_items = n_items
        self._n_bundle = bunlde_max_id + 1
        self._lambda_u = lambda_u
        self._lambda_i = lambda_i
        self._lambda_j = lambda_j
        self._lambda_d = lambda_d
        self._lambda_p = lambda_p
        self._lambda_bias = lambda_bias
        self._lambda_A = lambda_A
        self._learning_rate = learning_rate
        
        self._n_tags = n_tags
        self._configure_theano()
        self._generate_train_model_item_function()
        self._generate_test_model_function()

    def _configure_theano(self):
        theano.config.mode = 'FAST_RUN'
        theano.config.floatX = 'float32'
    
    def _generate_train_model_item_function(self):
        u = T.lvector('u')
        i = T.lmatrix('i')
        j = T.lmatrix('j')
        n1 = T.lvector('n1')#num of items in pos bundle
        n2 = T.lvector('n2')
        di = T.dvector('di')#cb
        dj = T.dvector('dj')#cb
        bdi = T.dvector('bdi')
        bdj = T.dvector('bdj')
        
        self.W1 = bpr_item.W #Pu
        self.H1 = theano.shared(H_item.astype('float32'), name='H')#Qi
        self.B1 = theano.shared(B_item.astype('float32'), name='B')#Bi
        self.TG1 = theano.shared(A_item.astype('float32'), name='TG')
        self.theta1 = bpr_item.X
        
        self.M1 = theano.shared(numpy.random.random((self._rank, self._rank)).astype('float64'), name='M1')#w
        self.M2 = theano.shared(numpy.random.random((self._rank, self._rank)).astype('float64'), name='M2')#u
        self.BDu = theano.shared(numpy.random.random((self._n_users)).astype('float64'), name='bdu')#u
        self.K = theano.shared(numpy.random.rand(), name='K')#k
        self.D = theano.shared(numpy.random.rand(), name='D')#C
        self.N = theano.shared(numpy.random.random(self._bundle_rank).astype('float32'), name='N')#Nb
        self.A1 = theano.shared(numpy.random.random((self._n_tags, self._n_tags)), name='A1')
        self.A2 = theano.shared(numpy.random.random((self._n_tags, self._n_tags)), name='A2')
                                
        
        x_ui = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[i].sum(axis=1).T/n1)).diagonal() + self.K*(self.B1[i].T/n1).T.sum(axis=1) + self.N[n1] + self.D*di + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[i].sum(axis=1).T/n1)).diagonal() + self.BDu[u] * bdi
        x_uj = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[j].sum(axis=1).T/n2)).diagonal() + self.K*(self.B1[j].T/n2).T.sum(axis=1) + self.N[n2] + self.D*dj + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[j].sum(axis=1).T/n1)).diagonal() + self.BDu[u] * bdj
        
        x_uij = T.nnet.sigmoid(x_ui-x_uj)
        obj = T.sum(T.log(x_uij) - self._lambda_u * (self.M1 ** 2).sum() - \
                    self._lambda_u * (self.M2 ** 2).sum()  - self._lambda_d * (self.K**2) - self._lambda_d * (self.D**2)\
                    -self._lambda_p * (self.N[n2]**2) - self._lambda_p * (self.N[n1]**2)) - self._lambda_A * (self.A1 ** 2).sum() - self._lambda_A * (self.A2 ** 2).sum()
        cost = - obj

        g_cost_M1 = T.grad(cost=cost, wrt=self.M1)
        g_cost_M2 = T.grad(cost=cost, wrt=self.M2)
        g_cost_K = T.grad(cost=cost, wrt=self.K)
        g_cost_N = T.grad(cost=cost, wrt=self.N)
        g_cost_D = T.grad(cost=cost, wrt=self.D)
        g_cost_A1 = T.grad(cost=cost, wrt=self.A1)
        g_cost_A2 = T.grad(cost=cost, wrt=self.A2)
        g_cost_BDu = T.grad(cost=cost, wrt=self.BDu)

        
        updates = [(self.M1, self.M1 - self._learning_rate * .001* g_cost_M1), (self.M2, self.M2 - self._learning_rate *.001* g_cost_M2), 
                   (self.K, self.K - self._learning_rate * .001*g_cost_K), (self.N, self.N - self._learning_rate *g_cost_N),
                  (self.D, self.D - self._learning_rate * g_cost_D), 
                   (self.A1, self.A1-self._learning_rate * .001 * g_cost_A1), (self.A2, self.A2-self._learning_rate * .001 * g_cost_A2),
                  (self.BDu, self.BDu-self._learning_rate * .001 * g_cost_BDu)]

        self.train_model_item = theano.function(inputs=[u, i, j, n1, n2, di, dj, bdi, bdj], outputs=cost, updates=updates)

    
    def train(self, s_users=None, s_pos_items=None, s_neg_items=None, s_pos_len=None, s_neg_len=None,
             s_pos_diversity=None, s_neg_diversity=None, bd_pos_discount = None, bd_neg_discount = None, batch_size=1000):
        
        if len(s_users) < batch_size:
            sys.stderr.write("WARNING: Batch size is greater than number of training samples, switching to a batch size of %s\n" % str(len(train_data)))
            batch_size = len(s_users)
        
        sgd_users, sgd_pos_items, sgd_neg_items = s_users, s_pos_items, s_neg_items
        n_sgd_samples = len(s_users)

        z = 0
        t2 = t1 = t0 = time.time()
        while (z+1)*batch_size < n_sgd_samples:
            
            self.train_model_item(
                sgd_users[z*batch_size: (z+1)*batch_size],
                sgd_pos_items[z*batch_size: (z+1)*batch_size],
                sgd_neg_items[z*batch_size: (z+1)*batch_size],
                s_pos_len[z*batch_size: (z+1)*batch_size],
                s_neg_len[z*batch_size: (z+1)*batch_size],
                s_pos_diversity[z*batch_size: (z+1)*batch_size],
                s_neg_diversity[z*batch_size: (z+1)*batch_size],
                bd_pos_discount[z*batch_size: (z+1)*batch_size],
                bd_neg_discount[z*batch_size: (z+1)*batch_size]
            )
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            sys.stderr.flush()
            t1 = t2
        if n_sgd_samples > 0:
            sys.stderr.write("\nTotal training time %.2f seconds; %e per sample\n" % (t2 - t0, (t2 - t0)/n_sgd_samples))
            sys.stderr.flush()
    
    def _generate_test_model_function(self):
        u = T.lvector('u')
        i = T.lmatrix('i')
        j = T.lmatrix('j')
        n1 = T.lvector('n1')
        n2 = T.lvector('n2')
        di = T.dvector('di')
        dj = T.dvector('dj')
        bdi = T.dvector('bdi')
        bdj = T.dvector('bdj')
        
        x_ui = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[i].sum(axis=1).T/n1)).diagonal() + self.K*(self.B1[i].T/n1).T.sum(axis=1) + self.N[n1] + self.D*di + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[i].sum(axis=1).T/n1)).diagonal() + self.BDu[u] * bdi
        x_uj = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[j].sum(axis=1).T/n2)).diagonal() + self.K*(self.B1[j].T/n2).T.sum(axis=1) + self.N[n2] + self.D*dj + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[j].sum(axis=1).T/n1)).diagonal() + self.BDu[u] * bdj
       
        x_uij = x_ui-x_uj - bdi + bdj
        self.test_model = theano.function(inputs=[u, i, j, n1, n2, di, dj, bdi, bdj], outputs=x_uij)
        
    def test_bundle(self, sgd_users, sgd_pos_items, sgd_neg_items, s_pos_len, s_neg_len,
                    s_pos_diversity, s_neg_diversity, bd_pos_discount, bd_neg_discount, batch_size=1000):
        
        auc_values = []
        z = 0
        t2 = t1 = t0 = time.time()
        n_sgd_samples = len(sgd_users)
        while (z+1)*batch_size < n_sgd_samples:
            pref_list=self.test_model(
                sgd_users[z*batch_size: (z+1)*batch_size],
                sgd_pos_items[z*batch_size: (z+1)*batch_size],
                sgd_neg_items[z*batch_size: (z+1)*batch_size],
                s_pos_len[z*batch_size: (z+1)*batch_size],
                s_neg_len[z*batch_size: (z+1)*batch_size],
                s_pos_diversity[z*batch_size: (z+1)*batch_size],
                s_neg_diversity[z*batch_size: (z+1)*batch_size],
                bd_pos_discount[z*batch_size: (z+1)*batch_size],
                bd_neg_discount[z*batch_size: (z+1)*batch_size]
            )
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            t1 = t2
            
            auc = np.sum([1.0 if a>0.0 else 0.0 for a in pref_list])
            auc /= batch_size
            
            auc_values.append(auc)
            sys.stderr.write("\rCurrent AUC mean (%s samples): %0.5f" % (str(z*batch_size), numpy.mean(auc_values)))
            sys.stderr.flush()
        
        sys.stderr.write("\n")
        sys.stderr.flush()
        return numpy.mean(auc_values)

In [25]:
print num_items

2515


In [26]:
bpr_bundle = BPR_Buncle1(10, max_bundle_size, len(user_bundle_map.keys()), num_items, \
                        len(sgd_tags.T), bunlde_max_id)

In [27]:
bpr_bundle.train(s_users=sgd_users, s_pos_items=sgd_pos_items, s_neg_items=sgd_neg_items, 
            s_pos_len=sgd_pos_len, s_neg_len=sgd_neg_len, s_pos_diversity=sgd_pos_diversity,
            s_neg_diversity=sgd_neg_diversity, \
            bd_pos_discount = sgd_pos_dicount, bd_neg_discount = sgd_neg_dicount)

Processed 2363000 ( 99.97% ) in 0.3696 seconds
Total training time 970.63 seconds; 4.106238e-04 per sample


In [29]:
# bpr_bundle.test_bundle(test_users, test_pos_items, test_neg_items, test_n1, test_n2,\
#                        test_pos_diversity, test_neg_diversity, \
#                        test_bd_pos_discount, test_bd_neg_discount)

In [63]:
print len(test_users)

1611611


In [ ]:
print max_bundle_size, len(user_bundle_map.keys()), num_items, len(tag_array.T)

In [40]:
import theano, numpy
import theano.tensor as T
import time
import sys
from collections import defaultdict

class BPR_Cold1(object):

    def __init__(self, rank, bundle_size, n_users, n_items, n_tags, lambda_u = 0.0025, lambda_i = 0.01, lambda_j = 0.0025, lambda_d = 0.0025, lambda_p = 0.00025, lambda_bias = 0.0, learning_rate = 0.05):
        
        self._rank = rank
        self._bundle_rank = bundle_size + 1 #only for penalty
        self._n_users = n_users
        self._n_items = n_items
        self._n_tags = n_tags
        self._lambda_u = lambda_u
        self._lambda_i = lambda_i
        self._lambda_j = lambda_j
        self._lambda_d = lambda_d
        self._lambda_p = lambda_p
        self._lambda_bias = lambda_bias
        self._learning_rate = learning_rate
        #self.bundle_discount = theano.shared(np.array(bundle_discount_array).astype('float32'), name='bundle_discount')
        self._configure_theano()
        self._generate_train_model_item_function()
        self._generate_test_model_function()

    def _configure_theano(self):
        theano.config.mode = 'FAST_RUN'
        theano.config.floatX = 'float32'
    
    def _generate_train_model_item_function(self):
        u = T.lvector('u')
        i = T.lmatrix('i')
        j = T.lmatrix('j')
        n1 = T.lvector('n1')#num of items in pos bundle
        n2 = T.lvector('n2')
        #di = T.dvector('di')#cb
        #dj = T.dvector('dj')#cb
        
        pu = numpy.random.random(self._n_users)
        
        self.W1 = bpr_item.W #Pu
        self.H1 = theano.shared(H_item.astype('float32'), name='H')#Qi
        self.B1 = theano.shared(B_item.astype('float32'), name='B')#Bi
        self.X = bpr_item.X
        self.A = theano.shared(A_item.astype('float32'), name='A')#Ai
        #self.Pu = theano.shared(pu.astype('float32'), name='Pu')
        
        self.M1 = theano.shared(numpy.random.random((self._rank, self._rank)).astype('float64'), name='M1')#w
        self.M2 = theano.shared(numpy.random.random((self._rank, self._rank)).astype('float64'), name='M2')#u
        self.E1 = theano.shared(numpy.random.random((self._n_tags,self._n_tags)).astype('float64'), name='E1')#theta
        self.E2 = theano.shared(numpy.random.random((self._n_tags,self._n_tags)).astype('float64'), name='E2')#alpha
        self.K = theano.shared(numpy.random.rand(), name='K')#k
        #self.D = theano.shared(numpy.random.rand(), name='D')#C
        self.N = theano.shared(numpy.random.random(self._bundle_rank).astype('float32'), name='N')#Nb
        
        #bi = self.bundle_discount[i]
        #bj = self.bundle_discount[j]
        
        x_ui = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[i].sum(axis=1).T/n1)).diagonal() + T.dot(T.dot(self.X[u],self.E1),T.dot(self.E2,self.A[i].sum(axis=1).T/n1)).diagonal() + self.K*(self.B1[i].T/n1).T.sum(axis=1) + self.N[n1] #+ self.D*di
        x_uj = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[j].sum(axis=1).T/n2)).diagonal() + T.dot(T.dot(self.X[u],self.E1),T.dot(self.E2,self.A[j].sum(axis=1).T/n2)).diagonal() + self.K*(self.B1[j].T/n2).T.sum(axis=1) + self.N[n2] #+ self.D*dj
        
        x_uij = T.nnet.sigmoid(x_ui-x_uj) #+ (self.Pu[u]) *(bi - bj)
        
        obj = T.sum(T.log(x_uij) - self._lambda_u * (self.M1 ** 2).sum() - \
                    self._lambda_u * (self.M2 ** 2).sum()  - self._lambda_d * (self.K**2) #- self._lambda_j * (self.D**2)\
                    - self._lambda_p * (self.N[n2]**2) - self._lambda_p * (self.N[n1]**2) - self._lambda_i * (self.E1 **2).sum() - \
                   self._lambda_i * (self.E2 ** 2).sum())# - self._lambda_j * (self.Pu ** 2))
        cost = - obj

        g_cost_M1 = T.grad(cost=cost, wrt=self.M1)
        g_cost_M2 = T.grad(cost=cost, wrt=self.M2)
        g_cost_K = T.grad(cost=cost, wrt=self.K)
        g_cost_N = T.grad(cost=cost, wrt=self.N)
        #g_cost_D = T.grad(cost=cost, wrt=self.D)
        g_cost_E1 = T.grad(cost=cost, wrt=self.E1)
        g_cost_E2 = T.grad(cost=cost, wrt=self.E2)
        
        
        updates = [(self.M1, self.M1 - self._learning_rate * .001* g_cost_M1), (self.M2, self.M2 - self._learning_rate *.001* g_cost_M2), 
                   (self.K, self.K - self._learning_rate * .001*g_cost_K), (self.N, self.N - self._learning_rate *g_cost_N),
                  #(self.D, self.D - self._learning_rate * g_cost_D), 
                   (self.E1, self.E1 - self._learning_rate *.001*g_cost_E1),(self.E2, self.E2 - self._learning_rate *.001 *g_cost_E2)]
                  

        self.train_model_item = theano.function(inputs=[u, i, j, n1, n2], outputs=cost, updates=updates)

    
    def train(self, s_users=None, s_pos_items=None, s_neg_items=None, s_pos_len=None, s_neg_len=None,
             s_pos_diversity=None, s_neg_diversity=None,batch_size=1000):
        
        if len(s_users) < batch_size:
            sys.stderr.write("WARNING: Batch size is greater than number of training samples, switching to a batch size of %s\n" % str(len(train_data)))
            batch_size = len(s_users)
        
        sgd_users, sgd_pos_items, sgd_neg_items = s_users, s_pos_items, s_neg_items
        n_sgd_samples = len(s_users)

        z = 0
        t2 = t1 = t0 = time.time()
        while (z+1)*batch_size < n_sgd_samples:
            
            self.train_model_item(
                sgd_users[z*batch_size: (z+1)*batch_size],
                sgd_pos_items[z*batch_size: (z+1)*batch_size],
                sgd_neg_items[z*batch_size: (z+1)*batch_size],
                s_pos_len[z*batch_size: (z+1)*batch_size],
                s_neg_len[z*batch_size: (z+1)*batch_size]
            )
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            sys.stderr.flush()
            t1 = t2
        if n_sgd_samples > 0:
            sys.stderr.write("\nTotal training time %.2f seconds; %e per sample\n" % (t2 - t0, (t2 - t0)/n_sgd_samples))
            sys.stderr.flush()
    
    def _generate_test_model_function(self):
        u = T.lvector('u')
        i = T.lmatrix('i')
        j = T.lmatrix('j')
        n1 = T.lvector('n1')
        n2 = T.lvector('n2')
        #di = T.dvector('di')
        #dj = T.dvector('dj')
        
        x_ui = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[i].sum(axis=1).T/n1)).diagonal() + T.dot(T.dot(self.X[u],self.E1),T.dot(self.E2,self.A[i].sum(axis=1).T/n1)).diagonal() + self.K*(self.B1[i].T/n1).T.sum(axis=1) + self.N[n1] #+ self.D*di
        x_uj = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[j].sum(axis=1).T/n2)).diagonal() + T.dot(T.dot(self.X[u],self.E1),T.dot(self.E2,self.A[j].sum(axis=1).T/n2)).diagonal() + self.K*(self.B1[j].T/n2).T.sum(axis=1) + self.N[n2] #+ self.D*dj
        #bi = self.bundle_discount[i]
        #bj = self.bundle_discount[j]
        x_uij = x_ui-x_uj #+ (self.Pu[u]) *(bi - bj)
        self.test_model = theano.function(inputs=[u, i, j, n1, n2], outputs=x_uij)
        
    def test_bundle(self, sgd_users, sgd_pos_items, sgd_neg_items, s_pos_len, s_neg_len, s_pos_diversity, s_neg_diversity, batch_size=1000):
        
        auc_values = []
        z = 0
        t2 = t1 = t0 = time.time()
        n_sgd_samples = len(sgd_users)
        while (z+1)*batch_size < n_sgd_samples:
            pref_list=self.test_model(
                sgd_users[z*batch_size: (z+1)*batch_size],
                sgd_pos_items[z*batch_size: (z+1)*batch_size],
                sgd_neg_items[z*batch_size: (z+1)*batch_size],
                s_pos_len[z*batch_size: (z+1)*batch_size],
                s_neg_len[z*batch_size: (z+1)*batch_size]
            )
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            t1 = t2
            
            auc = np.sum([1.0 if a>0.0 else 0.0 for a in pref_list])
            auc /= batch_size
            
            auc_values.append(auc)
            sys.stderr.write("\rCurrent AUC mean (%s samples): %0.5f" % (str(z*batch_size), numpy.mean(auc_values)))
            sys.stderr.flush()
        
        sys.stderr.write("\n")
        sys.stderr.flush()
        return numpy.mean(auc_values)

In [43]:
bpr_cold = BPR_Cold1(10, max_bundle_size, len(user_bundle_map.keys()), num_items, \
                        len(sgd_tags.T), bunlde_max_id)

In [ ]:
bpr_cold.train(s_users=sgd_users, s_pos_items=sgd_pos_items, s_neg_items=sgd_neg_items, 
          s_pos_len=sgd_pos_len, s_neg_len=sgd_neg_len, s_pos_diversity=sgd_pos_diversity, s_neg_diversity=sgd_neg_diversity)

Processed 949000 ( 40.15% ) in 0.5587 seconds

In [ ]:
bpr_cold.test_bundle(test_users, test_pos_items,
                     test_neg_items, test_n1, test_n2, test_pos_diversity, test_neg_diversity)